# Lanczos Spectrum demo
Minimal example on a tiny slice of WikiText-2.

In [2]:
import torch, matplotlib.pyplot as plt, numpy as np
from transformers import AutoModelForCausalLM
from datasets import load_dataset
from hessformer import run_lanczos, HvpMode

In [3]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model = AutoModelForCausalLM.from_pretrained(model_name,
    torch_dtype=torch.bfloat16, device_map="auto").eval()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [4]:
ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1%]")


In [ ]:
out = run_lanczos(model, ds, num_iter=10,
                  hvp_mode=HvpMode.REV_FWD,
                  batch_size=2)

HVP computation:   7%|██████████▏                                                                                                                                      | 13/184 [00:05<01:12,  2.37it/s]

In [ ]:
eigvals = out["spec"]["eigvals"].cpu()
gammas  = out["spec"]["gammas"].cpu()

plt.stem(eigvals, gammas, basefmt=" ")
plt.yscale("log")
plt.xlabel("Eigenvalue λ")
plt.ylabel("γ₀² (log)")
plt.title("Approx. Hessian spectrum")
plt.show()

You can also build a global histogram of the Lanczos-estimated eigen‐density.

In [ ]:
hist, bins = np.histogram(eigvals.numpy(), bins=50, weights=gammas.numpy())
plt.bar(bins[:-1], hist, width=bins[1]-bins[0])
plt.yscale('log')
plt.xlabel('λ')
plt.ylabel('density (log)')
plt.title('Lanczos density estimate')